### 3D Mesh Processing and Cristae Surface Extraction
Loads mitochondrial meshes, cleans them, and extracts cristae membranes with curvature mapping:

1. **Bounding box (BB):** Defines the 3-D region of interest.
2. **IBM/OMM cleanup:** Intersects IBM and OMM with BB and removes faces within 5 nm, yielding **IBS** and **OMS**.
3. **IMM curvature:** Loads IMM and attaches curvature values from *smooth_k1*.
4. **Cristae membrane (CMS):** Removes IMM faces within 5 nm of BB and 8 nm of IBS while preserving curvature.
5. **High-curvature regions:** Segments CMS vertices with curvature >80 µm⁻¹.
6. **Outputs:** Prints mitochondrial volume, total cristae area, high-curvature area, and shows curvature-colored 3-D visualizations.


In [ ]:
from MeshOperations3D import process, visualize
import os
#import numpy as np
import pyvista as pv
import trimesh
# ==============================================================
# === Processing Pipeline
# ==============================================================
base_dir = r"./Demo Images and Meshes/control 4"

# --- 1. Load and prep bounding box mesh
BB = process.pyvista_to_trimesh(pv.read(os.path.join(base_dir, "BB_remeshed.stl")))

# --- 2. IBM preprocessing: boolean intersect + cleanup
IBM = process.pyvista_to_trimesh(pv.read(os.path.join(base_dir, "IBM_remeshed.stl")))
IBM = trimesh.boolean.intersection([IBM, BB], engine="manifold")
IBS = process.remove_close_faces(IBM, BB, threshold=0.005)

# --- 3. OMM preprocessing: boolean intersect + cleanup
OMM = process.pyvista_to_trimesh(pv.read(os.path.join(base_dir, "OMM_remeshed.stl")))
OMM = trimesh.boolean.intersection([OMM, BB], engine="manifold")
OMS = process.remove_close_faces(OMM, BB, threshold=0.005)

# --- 4. IMM mesh and curvature
IMM_pv = pv.read(os.path.join(base_dir, "IMM.stl"))
BB_pv = pv.read(os.path.join(base_dir, "BB_remeshed.stl"))  
data_array = process.load_pickle(os.path.join(base_dir, "smooth_k1"))[0]
IMM_pv.point_data['Curvature'] = data_array

# --- 5. Clean IMM against BB and IBS
CMS, new_curvature = process.remove_close_faces_with_curvature_pv(IMM_pv, BB_pv, threshold=0.005)
IBS_pv = process.trimesh_to_pyvista(IBS)
CMS, new_curvature = process.remove_close_faces_with_curvature_pv(CMS, IBS_pv, threshold=0.008)

# --- 6. Separate high-curvature surface
CMS_high_pv, CMS_low_pv = process.separate_high_curvature_mesh(CMS, new_curvature, 80)
print(f"Mito volume: {OMM.volume:.3f} µm³" if OMM else "Mito volume: 0 µm³")
print(f"Cristae surface area: {CMS.area:.3f} µm²" if CMS else "Cristae surface area: 0 µm²")
print(f"High-curvature surface area: {CMS_high_pv.area:.3f} µm²" if CMS_high_pv else "High-curvature surface area: 0 µm²")


# --- 7. Visualize results
visualize.display_cristae(OMS, IBS, CMS, new_curvature, threshold=0, smoothing_sigma=1)
visualize.display_curvature_colored_meshes(OMS, IBS_pv, CMS, new_curvature,
                                 min_val=0, max_val=80, smoothing_sigma=0.1)

### Cristae Junction Detection and Quantification
Identifies and measures cristae junctions within the mitochondrial inner membrane:

1. **Cleanup:** Removes IMM and IBM faces near the bounding box and splits IMM into **CMS_tr** (cristae) and **IBS_tr** (inner boundary).
2. **Junction detection:** Finds shared edges between CMS_tr and IBS_tr and filters valid junction centroids.
3. **Metrics:** Computes the total number of cristae junctions and their density (per µm³ of OMM).
4. **Visualization:** Renders OMS, IBS_tr, CMS_tr, and detected junction centroids in 3-D.


In [ ]:
import numpy as np
# --- Cristae junction analysis ---
IMM = process.pyvista_to_trimesh(IMM_pv)
IMM = process.remove_close_faces(IMM, BB, threshold=0.005)

IB_tr = process.remove_close_faces(IBM, BB, threshold=0.005)
CMS_tr, IBS_tr = process.separate_close_faces(IMM, IB_tr, threshold=0.008)

edge_actor, centroids, centroid_actor = process.shared_edge_junctions(CMS_tr, IBS_tr)
refined_centroids, refined_centroid_actor = process.filter_junctions(CMS_tr, np.array(centroids))

# Cristae junction metrics
cristae_junction_count   = len(refined_centroids)
cristae_junction_density = cristae_junction_count / OMM.volume  # per µm³

print(f"Cristae junctions: {cristae_junction_count}")
print(f"Cristae junction density: {cristae_junction_density} per µm³")

# Visualization
visualize.display_junctions(OMS, IBS_tr, CMS_tr, refined_centroids)

### ER–Mitochondria Contacts and Ribosome Segmentation
Processes endoplasmic reticulum (ER) and ribosome meshes to map ER–mitochondria contacts and nearby ribosomes:

1. **Mesh cleanup:** Intersects ER and ribosome meshes with the bounding box (BB) and removes faces within 5 nm.  
2. **ERMCS extraction:** Isolates ER faces within 20 nm of the OMM to define **ER–mitochondria contact sites (ERMCS)**.  
3. **Ribosome proximity:** Separates ribosome faces within 10 nm of the OMM (**Ribo_near**).  
4. **Outputs:** Prints ERMCS and total ER surface areas (µm²) and displays a 3-D visualization of OMS, IBS, CMS, ER, ERMCS, and ribosomes.


In [ ]:

# Load ER and Ribosome meshes
ER = process.pyvista_to_trimesh(pv.read(os.path.join(base_dir, "ER_remeshed.stl")))
Ribo = process.pyvista_to_trimesh(pv.read(os.path.join(base_dir, "Ribo_remeshed.stl")))

# Boolean operations & cleanup
ER = trimesh.boolean.intersection([ER, BB], engine="manifold")
Ribo = trimesh.boolean.intersection([Ribo, BB], engine="manifold")

ER = process.remove_close_faces(ER, BB, threshold=0.005)
ER, ERMCS = process.separate_close_faces(ER, OMM, threshold=0.02) #ERMCS = ER within 20 nm of OMM
Ribo_far, Ribo_near = process.separate_close_faces(Ribo, OMM, threshold=0.01)

print(f"ERMCS surface area: {ERMCS.area:.3f} µm²" if ERMCS else "ERMCS surface area: 0 µm²")
print(f"ER surface area: {ER.area:.3f} µm²" if ER else "ER surface area: 0 µm²")


# Visualization
visualize.display_all(OMS, IBS, CMS, ER, ERMCS, Ribo,
            new_curvature, threshold=0, smoothing_sigma=1)


### Ribosome Clustering and Proximity Analysis
Identifies and analyzes ribosome clusters near mitochondria:

1. **Cluster merging:** Splits the ribosome mesh into connected components and merges those closer than 20 nm to form **ribosome clusters**.  
2. **Visualization:** Displays OMS, IBS, CMS, ER, ERMCS, and all ribosome clusters with CMS curvature overlay.  
3. **Proximity analysis:** Computes each cluster’s minimum surface distance and centroid distance to the OMS and reports ribosome volumes and OMS surface area (all in nm-based units).


In [ ]:
# Convert your Trimesh ribosome mesh to PyVista clusters
ribosome_clusters = process.separate_ribosomes_with_merge(Ribo)
# Show clusters with other meshes
visualize.display_ribosome_clusters(OMS, IBS, CMS, ER, ERMCS, ribosome_clusters,
                                   new_curvature, threshold=80,
                                   smooth_boundaries=False, smoothing_sigma=1.0)
# Analyze vs OMS
process.analyze_ribosomes_vs_OMS(ribosome_clusters, OMS)